In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from itemrep_inference import topk_distance

artifact_path = 'artifacts'

umap = pickle.load(open(os.path.join(artifact_path,"usermap.p"), "rb"))
imap_inverse = pickle.load(open(os.path.join(artifact_path,"itemmap_inv.p"), "rb"))
interactions = pd.read_pickle(os.path.join(artifact_path,"interactions.p"))

In [2]:
def recommend(userid, topk=2):
    uid = umap[userid]
    _temp = interactions.iloc[uid]
    _temp = _temp[_temp!=0]
    _tempdf = pd.DataFrame(columns=['itemid','distance'])
    for row in _temp.iteritems():
        _temp1 = topk_distance(imap_inverse[row[0]])
        _temp2 = pd.DataFrame(list(_temp1.items()), columns=['itemid','distance'])
        _temp2['weight'] = row[1]
        _tempdf = _tempdf.append(_temp2)
    _tempdf = _tempdf[_tempdf['distance']!=0]
    _tempdf['distance']+=1
    _tempdf['score'] = np.sqrt(_tempdf['weight'])/np.log(_tempdf['distance'])
    _tempdf = _tempdf.set_index('itemid')
    _tempdf = _tempdf[['score']].groupby(['itemid']).mean()
    _tempdf = _tempdf.sort_values(by='score', ascending=False)
    return _tempdf.index.values[:topk]

In [3]:
recommend('user1')

array(['10003', '10002'], dtype=object)

In [11]:
from fastapi import FastAPI
from typing import Optional

app = FastAPI()

@app.get("/")
async def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
async def read_item(item_id: int, q: Optional[str] = None):
    return {"item_id": item_id, "q": q}

In [ ]:
from fastapi import FastAPI
from typing import Optional

app = FastAPI()

@app.get("/recommend/{userid}")
async def read_item(userid: str):
    recs = json.dumps(recommend(userid).tolist())
    return recs